In [1]:
import tensorflow as tf
import keras

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)


2024-01-26 18:04:28.045565: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-26 18:04:28.088848: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-26 18:04:28.088881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-26 18:04:28.089975: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-26 18:04:28.097356: I tensorflow/core/platform/cpu_feature_guar

ImportError: cannot import name 'cast' from partially initialized module 'keras.src.backend' (most likely due to a circular import) (/home/ryan/.local/lib/python3.10/site-packages/keras/src/backend/__init__.py)

In [1]:

import numpy as np
import os
import time


In [2]:
import tensorflow as tf
from tensorflow import keras

2024-01-26 17:51:08.078238: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-26 17:51:08.120109: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-26 17:51:08.120135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-26 17:51:08.121163: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-26 17:51:08.127918: I tensorflow/core/platform/cpu_feature_guar

In [3]:
text = open('roberts_arguments.txt', 'rb').read().decode(encoding='utf-8')

In [4]:
print(f'Length of text: {len(text)} characters')

Length of text: 284181 characters


In [5]:
print(text[:250])

We'll hear argument next in Delaware versus Pennsylvania and Wisconsin and the consolidated case. Mr. Katyal. ORAL ARGUMENT OF NEAL K. KATYAL ON BEHALF OF DELAWAREMr. --I'm sorry, go ahead.How -- how much -- what's the comparison in terms of total va


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

66 unique characters


In [7]:
# Vectorize the text without using StringLookup
vocab = sorted(set(text))
char_to_id = {char: idx for idx, char in enumerate(vocab)}
id_to_char = np.array(vocab)

# Convert characters to IDs
ids = np.array([char_to_id[char] for char in text])

# Convert IDs back to characters
chars = id_to_char[ids]

# Now, 'ids' and 'chars' can be used as needed


In [8]:

ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)


ImportError: Keras cannot be imported. Check that it is installed.

In [ ]:
import tensorflow as tf
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)


In [9]:
import keras
print(keras.__version__)


ImportError: cannot import name 'cast' from partially initialized module 'keras.src.backend' (most likely due to a circular import) (/home/ryan/.local/lib/python3.10/site-packages/keras/src/backend/__init__.py)

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()


In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?
Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids